In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from time import time
requests = 0
from IPython.core.display import clear_output
from warnings import warn

In [2]:

def writ(writer):
    k = []
    for w in writer:
        if (w.find('h4').getText() == "Writers:"):
            writ = w.find_all('a')
            for i in writ:
                k.append(i.text) 
    return k

def mov_gross(release):
    g = []
    for c in release:
        heading = c.find('h4')
        if heading is not None:
            if (c.find('h4').getText() == "Cumulative Worldwide Gross:"):
                g = c.find('h4').next_sibling.rstrip()
    return g

def mov_rel(release):
    r = []
    for c in release:
        heading = c.find('h4')
        if heading is not None:
            if (c.find('h4').getText() == "Release Date:"):
                r = c.find('h4').next_sibling.rstrip()
    return r

def mov_bud(release):
    b = []
    for c in release:
        heading = c.find('h4')
        if heading is not None:
            if (c.find('h4').getText() == "Budget:"):
                b = c.find('h4').next_sibling.rstrip()
    return b
    
    

def inside_movie(link):
    response1 = get('http://www.imdb.com/' + link + '?ref_=adv_li_tt')
    page_html_in = BeautifulSoup(response1.text, 'html.parser')
    movie_detail = page_html_in.find_all('div', class_ = 'flatland')
    for sing_movie in movie_detail:
        writer = sing_movie.find_all('div', {'class':'credit_summary_item'})
        wr = writ(writer)
        if wr is None:
            writers.append(np.nan)
        else:
            writers.append(wr)
        
        release = sing_movie.find_all('div', {'class':'txt-block'})
        gros = mov_gross(release)
        rel = mov_rel(release)
        bud = mov_bud(release)
        if gros is None:
            cum_gross.append(np.nan)
        else:
            cum_gross.append(gros)
        if rel is None:
            release_d.append(np.nan)
        else:
            release_d.append(rel)
        if bud is None:
            budget.append(np.nan)
        else:
            budget.append(bud)
            
years_url = [str(i) for i in range(2001,2020)]

pages = ["1", "51", "101", "151", "201", "251"]

start_time = time()
requests = 0

Title = []
Year = []
Rating = []
Metascore = []
Votes = []
Revenue = []
Genre = []
Time = []
Director = []
Stars = []
Description = []
writers = []
cum_gross = []
budget = []
release_d = []

# For every year in the interval 2001-2019
for year_url in years_url:

    # For every page in the interval 1-5
    for page in pages:

    
        response = get('http://www.imdb.com/search/title?release_date=' + year_url + '&sort=num_votes,desc&start=' + page)
        
        sleep(randint(8,15))

        requests += 1
        
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests))

        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        if requests > 1000:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        movie_container = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for first_movie in movie_container:
            link = first_movie.find('a').get('href')
            inside_movie(link)

            Title.append(first_movie.h3.a.text)
            year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold')
            Year.append(year.text.strip())
            Rating.append(float(first_movie.strong.text))
            try:
                metascore = first_movie.find('span', class_ = 'metascore favorable')
                Metascore.append(int(metascore.text))
            except:
                Metascore.append(np.nan)
        
            movieNumbers = first_movie.find_all('span', attrs = {'name':'nv'})
            if len(movieNumbers) == 2:
                Votes.append(movieNumbers[0].text)
                Revenue.append(movieNumbers[1].text)
            elif len(movieNumbers) == 1:
                Votes.append(movieNumbers[0].text)
                Revenue.append(np.nan)
            else:
                Votes.append(np.nan)
                Revenue.append(np.nan)
            try:
                time = first_movie.find('span', class_ = 'runtime')
                Time.append(time.text)
            except:
                Time.append(np.nan)
            try:
                genre = first_movie.find('span', class_ = 'genre')
                Genre.append(genre.text.strip())
            except:
                Genre.append(np.nan)
                
            movieCast = first_movie.find("p", class_="")
            try:
                cast = movieCast.text.replace("\n","").split('|')
                cast = [x.strip() for x in cast]
                cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
                Director.append(cast[0])
                Stars.append([x.strip() for x in cast[1].split(",")])
            except:
                cast = movieCast.text.replace("\n","").strip()
                Director.append(np.nan)
                Stars.append([x.strip() for x in cast.split(",")])
        
            Description.append(first_movie.find_all("p", class_="text-muted")[-1].text.lstrip())
            


df = pd.DataFrame({'Title':Title,'Year':Year,'Rating':Rating,'Metascore':Metascore,'Votes':Votes,'Revenue':Revenue,'Genre':Genre,'Time':Time, 'Director':\
      Director, 'Star':Stars, 'Description':Description, 'Writers':writers, 'Gross':cum_gross, 'Release':release_d, 'Budget':budget}) 

df.to_csv('imdb_multi.csv', index=False, encoding='utf-8')

print(df)



                                                  Title               Year  \
0     The Lord of the Rings: The Fellowship of the Ring             (2001)   
1                                      A Beautiful Mind             (2001)   
2                                        Monsters, Inc.             (2001)   
3                                          Donnie Darko             (2001)   
4                                                Amélie             (2001)   
...                                                 ...                ...   
5695                             Bill Burr: Paper Tiger  (2019 TV Special)   
5696                                          Miss Bala             (2019)   
5697                                               Dark           (2017– )   
5698                                        The Kitchen             (2019)   
5699                      The Accidental Prime Minister             (2019)   

      Rating  Metascore      Votes   Revenue                   